In [1]:
import pandas as pd

In [29]:
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
test_id=test["PassengerId"]
test_name=test["Name"]
data.head(5)
#test.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
#create a function to clean data 
def clean(data):
    #dropping unnecessary columns
    data=data.drop(["Name","Ticket","Cabin","PassengerId"],axis=1 )
    # replacing blanks 
    cols=["SibSp", "Parch", "Age", "Fare"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    data['Embarked'] = data['Embarked'].fillna("U")
    return data
data=clean(data)
test=clean(test)
data.head(100)

C:\Users\shail\AppData\Local\Temp\ipykernel_13708\4153167119.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
C:\Users\shail\AppData\Local\Temp\ipykernel_13708\4153167119.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
95,0,3,male,28.0,0,0,8.0500,S
96,0,1,male,71.0,0,0,34.6542,C
97,1,1,male,23.0,0,1,63.3583,C
98,1,2,female,34.0,0,1,23.0000,S


In [49]:
#Label encoding of sex and embarked data 
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
cols=["Sex","Embarked"]
for col in cols:
    data[col]=le.fit_transform(data[col])
    test[col]=le.transform(test[col])
    print(le.classes_)
data.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [5]:
#Using logistic regression Model
#instead you could also do linear regression, decision tree, k-nearest
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
y=data["Survived"]
x= data.drop("Survived" ,axis=1)
x_train,x_val,y_train,y_val=train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
#classification
cls=LogisticRegression(random_state=0, max_iter=1000).fit(x_train, y_train)

In [7]:
predictions=cls.predict(x_val)
from sklearn.metrics import accuracy_score 
accuracy_score(y_val,predictions)

0.8100558659217877

In [8]:
#predictions 
submission_pred= cls.predict(test)

In [31]:
df=pd.DataFrame({"PassengerId":test_id.values,"Name":test_name.values,  "Survived": submission_pred,})

In [33]:
df.to_csv("results.csv" , index=False)

In [87]:
#Using Random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
y=data["Survived"]
x= data.drop("Survived" ,axis=1)
x_train,x_val,y_train,y_val=train_test_split(x, y, test_size=0.2, random_state=42)

In [89]:
rf_cls=RandomForestClassifier(n_estimators=100, random_state=42).fit(x_train, y_train)

In [91]:
from sklearn.metrics import mean_squared_error, r2_score
predictions=rf_cls.predict(x_val)
from sklearn.metrics import accuracy_score 
accuracy_score(y_val,predictions)

0.8212290502793296

In [93]:
submission_pred_rfc= rf_cls.predict(test)

In [105]:
df=pd.DataFrame({"PassengerId":test_id.values,"Survived": submission_pred_lr,})

In [107]:
df.to_csv("results.csv" , index=False)

In [ ]:
#random forest classifier has better accuracy